In [60]:
import pandas as pd

def load_data(file_path):
    """
    Load data from all sheets in an Excel file into a single DataFrame.
    
    Parameters:
    - file_path (str): The path to the Excel file.
    
    Returns:
    - pd.DataFrame: A DataFrame containing all data from the Excel file.
    """
    # Concatenate all sheets read from the Excel file into a single DataFrame, ignoring the index to prevent duplication.
    return pd.concat(pd.read_excel(file_path, sheet_name=None), ignore_index=True)

def remove_unwanted_columns(df, columns_to_delete):
    """
    Remove specified columns from the DataFrame that are not needed.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame from which to remove columns.
    - columns_to_delete (list of str): A list of column names to be removed.
    
    Returns:
    - pd.DataFrame: The DataFrame after removing specified columns.
    """
    # Remove columns by filtering out those in the columns_to_delete list if they exist in the DataFrame.
    df.drop(columns=[col for col in columns_to_delete if col in df.columns], inplace=True)
    return df

def convert_to_categorical(df, categorical_columns):
    """
    Convert specified columns in the DataFrame to categorical data type if they exist.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame to modify.
    - categorical_columns (list of str): Columns to convert to categorical type.
    
    Returns:
    - pd.DataFrame: The DataFrame after conversion.
    """
    # Iterate over each column in the list; convert to category type if present in DataFrame.
    for col in categorical_columns:
        if col in df.columns:
            df[col] = df[col].astype('category')
    return df

def rename_columns(df, rename_dict):
    """
    Renames columns in the DataFrame according to a provided dictionary.

    Parameters:
    - df (pd.DataFrame): The DataFrame whose columns need to be renamed.
    - rename_dict (dict): A dictionary mapping old column names to new column names.

    Returns:
    - pd.DataFrame: The DataFrame with renamed columns.
    """
    # Rename columns using the provided dictionary
    df.rename(columns=rename_dict, inplace=True)
    return df

def transform_cal_code(df):
    """
    Transforms values in the 'Cal Code' column of the DataFrame according to specified rules:
    - 'N01', 'N03', 'N26' are changed to 'Night'.
    - 'I01', 'I26' are changed to 'Internet'.
    - All other values are changed to 'Day'.

    Parameters:
    - df (pd.DataFrame): The DataFrame with the 'Cal Code' column to be transformed.

    Returns:
    - pd.DataFrame: The DataFrame after the 'Cal Code' transformations.
    """
    # Define a mapping for specific Cal Code transformations
    code_map = {
        3: 'Night',
        'N01': 'Night',
        'N03': 'Night',
        'N26': 'Night',
        'I01': 'Internet',
        'I26': 'Internet'}
        # Apply the map to the 'Cal Code' column, defaulting to 'Day' for codes not in the map
    df['Student Type'] = df['Student Type'].map(code_map).fillna('Day')
    return df

def transform_ent(df):
    """
    Transforms values in the 'Cal Code' column of the DataFrame according to specified rules:
    - 'N01', 'N03', 'N26' are changed to 'Night'.
    - 'I01', 'I26' are changed to 'Internet'.
    - All other values are changed to 'Day'.

    Parameters:
    - df (pd.DataFrame): The DataFrame with the 'Cal Code' column to be transformed.

    Returns:
    - pd.DataFrame: The DataFrame after the 'Cal Code' transformations.
    """
    # Define a mapping for specific Cal Code transformations
    code_map = {
        1: 'Norwood',
        2: 'Coleman',
        3: 'South Austin',
        4: 'Billy Moore',
        5: 'Bradshaw',
        6: 'AYW',
        7: 'Diboll',
        8: 'Connally',
        800: 'Alumni'}
        # Apply the map to the 'Cal Code' column, defaulting to 'Day' for codes not in the map
    df['School'] = df['School'].map(code_map).fillna('')
    return df

def export_to_excel(df, output_path):
    """
    Exports the DataFrame to an Excel file.

    Parameters:
    - df (pd.DataFrame): The DataFrame to export.
    - output_path (str): The path where the Excel file will be saved.

    Returns:
    None
    """
    # Write the DataFrame to an Excel file at the specified output path
    df.to_excel(output_path, index=False)

In [61]:

file_path = "output_data/Project_Data.xlsx"
output_path = "output_data/Cleaned_Project_Data.xlsx"
columns_to_delete = ['Status', 'Age', 'His/Lat Ethnicity', 'Fed Race Description', 
                         'Home Language Description', 'Econ Dis', 'Qualifies as Eco Dis', 'Class', 'Withdrw', 'Cat']
categorical_columns = ['Sex', 'Local Race', 'Marital Status', 'Student Language Description', 
                           'Economic Disadvantage', 'Food Stamps/TANF', 'Term Ltrl', 'Sub', 'Class Status', 
                           'Cat', 'Description', 'Grade']
        # Dictionary for renaming columns
rename_dict = {'Student Language Description': 'Student Language', 'Schl Yr': 'School Year', 'DOB': 'Date of Birth', 'Other ID': 'ID', 'Ent': 'School', 'Cal Code': 'Student Type'}   
try:
    df = load_data(file_path)
    clean_df = remove_unwanted_columns(df, columns_to_delete)
    clean_df = rename_columns(clean_df, rename_dict)
    clean_df = convert_to_categorical(clean_df, categorical_columns)
    clean_df = transform_cal_code(clean_df)
    clean_df = transform_ent(clean_df)
    print(clean_df.head())
except Exception as e:
    print(f"An error occurred: {e}")




    School Student Type       ID Sex              Local Race Grad Date  \
0  Norwood          Day      837   F      WHITE/NON HISPANIC       NaT   
1  Norwood          Day  2276777   M      BLACK/NON HISPANIC       NaT   
2  Norwood          Day      344   M      BLACK/NON HISPANIC       NaT   
3  Norwood          Day  4576892   F      WHITE/NON HISPANIC       NaT   
4  Norwood          Day      872   F  ASIAN/PACIFIC ISLANDER       NaT   

  Date of Birth Birth Country Student Language Marital Status  ... Entity  \
0    1992-01-17          Iraq           ARABIC        Married  ...    NaN   
1    1987-11-30         Sudan           ARABIC        Married  ...    NaN   
2    1988-01-01       Somaila           SOMALI         Single  ...    NaN   
3    1994-09-07   Afghanistan           PASHTO        Married  ...    NaN   
4    1992-11-10           NaN            FARSI            NaN  ...    NaN   

  School Year  Term Ltrl  Sub  Class Status  Earned Cred Description Grade  \
0         NaN 

In [51]:
 #Grouping by 'Other ID' and aggregating the data
aggregated_df = clean_df.groupby('ID').agg({
    'Earned Cred': 'sum',  # Sum of credits earned
    'Date of Birth': 'min',  # Earliest date of birth
    'Sex': lambda x: x.mode()[0] if not x.mode().empty else None,
    'Marital Status': lambda x: x.mode()[0] if not x.mode().empty else None,
    # Add other columns as needed
}).reset_index()

aggregated_df.head()

,ID,Earned Cred,Date of Birth,Sex,Marital Status
0,0,0.0,2000-07-29,F,None
1,1,32.0,1977-11-11,F,Married
2,2,1.5,1988-05-20,M,None
3,3,22.0,1978-04-08,F,Single
4,4,24.5,1983-07-03,F,Single


In [63]:
export_to_excel(clean_df, output_path)